In [ ]:
from random import *

In [24]:
class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté
    dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """
    
    
    def __init__(self, height : int, width : int, empty : bool = False):
        """
        Constructeur d'un labyrinthe de height cellules de haut
        et de width cellules de large
        Les voisinages sont initialisés à des ensembles vides quand empty vaut False
        et les voisinages comportent tout les voisins qui lui sont contigues dans la grille quand empty vaut True
        Remarque : empty vaut False par defaut et donc chaque cellule est completement emmuree par defaut.
        
        @param height : nombre de cases en hauteur
        @param width : nombre de cases en largeur
        @param empty : True si le labyrinthes est vide (pas de murs) et False si le labyrinthes est plein
        (tout les murs), empty vaut False par defaut
        """
        
        self.height    = height
        self.width     = width
        self.empty     = empty
        if empty: # creation d'un labyrinthe sans mur entre les differents sommets
            self.neighbors = {(x,y): {(x-1,y),(x,y-1),(x+1,y),(x,y+1)} for x in range(height) for y in range (width)}
            for x in range(height):
                self.neighbors[x,width-1].remove((x,width))
        else: # creation d'un labyrinthe ou les differents sommets n'ont pas de voisins (tout les murs sont presents)
            self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}
  

    def info(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = "**Informations sur le labyrinthe**\n"
        txt += f"- Dimensions de la grille : {self.height} x {self.width}\n"
        txt += "- Voisinages :\n"
        txt += str(self.neighbors)+"\n"
        valid = True
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            for c2 in self.neighbors[c1]:
                if c1 not in self.neighbors[c2]:
                    valid = False
                    break
            else:
                continue
            break
        txt += "- Structure cohérente\n" if valid else f"- Structure incohérente : {c1} X {c2}\n"
        return txt

    
    def __str__(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"

        return txt
        
        
    def add_wall(self, c1 : tuple, c2 : tuple) -> None:
        """
        Methode d'instance Maze permettant d'ajouter un mur entre deux cellules c1 et c2.
        
        @param c1 premiere cellule (format (x : int, y: int))
        @param c2 seconde cellule  (format (x : int, y: int))
        @return rien
        """
        
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        
        # Ajout du mur
        if c2 in self.neighbors[c1]:      # Si c2 est dans les voisines de c1
            self.neighbors[c1].remove(c2) # on le retire
        if c1 in self.neighbors[c2]:      # Si c3 est dans les voisines de c2
            self.neighbors[c2].remove(c1) # on le retire
        
        return None

    
    def remove_wall(self,c1,c2)-> None:
        """
        Methode d'instance Maze permettant de supprimer un mur entre deux cellules c1 et c2.
        
        @param c1 premiere cellule (format (x : int, y: int))
        @param c2 seconde cellule  (format (x : int, y: int))
        @return rien
        """
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # retirer le mur
        if c2 not in self.neighbors[c1]:
            self.neighbors[c1].add(c2)
        if c1 not in self.neighbors[c2]:
            self.neighbors[c2].add(c1)
        
        return None


    def get_walls(self)-> list:
        """
        Methode d'instance Maze permettant de recuperer la liste des murs presents dans le labyrinthe.
        
        @return la liste des murs presents dans le labyrinthe (format [[(x1, y1), (x2, x2)], [(x1, y1), (x3, y3)], ...])
        """
        # initialisation de la liste des murs presents
        liste_mur=[]
        
        # parcours du labyrinthe
        for i in range(self.height):
            for j in range(self.width):
                # si il y a une cellule au dessus non voisine
                if (i+1,j) not in self.neighbors[i,j] and (i+1<self.height):
                    # alors il y a une mur
                    mur = [(i,j),(i+1,j)]
                    # si ce mur n'est pas deja recense
                    if mur not in liste_mur:
                        # on ajoute le mur a la liste
                        liste_mur.append(mur)
                # si il y a une cellule a droite non voisine
                if (i,j+1) not in self.neighbors[i,j] and (j+1<self.width):
                    # alors il y a un mur
                    mur = [(i,j),(i,j+1)]
                    # si ce mur n'est pas deja recense
                    if mur not in liste_mur:
                        # on ajoute le mur a la liste
                        liste_mur.append(mur)
                        
        # on renvoie la liste
        return liste_mur
    
    
    def fill(self)-> None:
        """
        Methode d'instance Maze permettant d'ajouter tout les murs non presents dans le labyrinthe.
        Remarque : les murs deja places sont ignores par add_wall()
        
        @return rien
        """
        
        # pour chaque emplacement de mur (parcours)
        for i in range(self.height):
            for j in range(self.width):
                
                # si on peut encore placer des murs en bas et a droite de la cellule 
                if i < (self.height-1) and j < (self.width-1):
                    # on tente de placer un mur en bas de la cellule
                    self.add_wall((i, j), (i+1, j))
                    # on tente de placer un mur a droite de la cellule
                    self.add_wall((i, j), (i, j+1))
                
                # sinon si on ne peut plus placer un mur en bas
                elif i == (self.height-1) and j < (self.width-1):
                    # on tente de placer un mur à droite de la cellule
                    self.add_wall((i, j), (i, j+1))
                
                # sinon si on ne peut plus placer un mur a droite
                elif i < (self.height-1) and j == (self.width-1):
                    # on tente de placer un mur en bas de la cellule
                    self.add_wall((i, j), (i+1, j))
                    
        return None
    
    def to_empty(self):
        """
        Methode d'instance Maze permettant de retirer tout les murs dans le labyrinthe.
        Remarque : les murs deja absents sont ignores par remove_wall()
        Remarque 2 : l'attribut empty existe deja
        
        @return rien
        """
        # pour chaque emplacement de mur (parcours)
        for i in range(self.height):
            for j in range(self.width):
                
                # si on peut encore retirer des murs en bas et a droite de la cellule 
                if i < (self.height-1) and j < (self.width-1):
                    # on tente de retirer un mur en bas de la cellule
                    self.remove_wall((i, j), (i+1, j))
                    # on tente de retirer un mur a droite de la cellule
                    self.remove_wall((i, j), (i, j+1))
                
                # sinon si on ne peut plus retirer un mur en bas
                elif i == (self.height-1) and j < (self.width-1):
                    # on tente de retirer un mur à droite de la cellule
                    self.remove_wall((i, j), (i, j+1))
                
                # sinon si on ne peut plus retirer un mur a droite
                elif i < (self.height-1) and j == (self.width-1):
                    # on tente de retirer un mur en bas de la cellule
                    self.remove_wall((i, j), (i+1, j))
                    
        return None
    
    
    def get_contiguous_cells(self, c: tuple) -> list:
        """
        Methode d'instance Maze permettant de recuperer la liste des cellules 
        contigues a c dans la grille (sans s’occuper des eventuels murs)
        Remarque : la liste retournee ne peut pas contenir plus de quatre voisins
        
        @param c : cellule a utiliser (format (x, y))0
        @return : liste des voisins contigues de c (format [(x1, y1), (x2, y2), ...])
        """
        
        # on recupere les coordonnees de c
        i, j = c
        # on initialise la liste pour stocker les voisins
        voisins = []
        
        # si il peut y avoir un voisin en haut
        if i > 0:
            # on ajoute la cellule en haut aux voisins
            voisins.append((i-1, j))
        # si il peut y avoir un voisin en bas
        if i < self.height-1:
            # on ajoute la cellule en bas aux voisins
            voisins.append((i+1, j))
        # si il peut y avoir un voisin a gauche
        if j > 0:
            # on ajoute la cellule de gauche aux voisins
            voisins.append((i, j-1))
        # si il peut y avoir un voisin a droite
        if j < self.width-1:
            # on ajoute le voisin de droite aux voisins
            voisins.append((i, j+1))
        
        # on renvoie la liste des voisins
        return voisins
    
    
    def get_reachable_cells(self, c : tuple) -> list:
        """
        Methode d'instance Maze permettant de recuperer la liste des cellules accessibles depuis c 
        (c’est-a-dire les cellules contigues à c qui sont dans le voisinage de c)
        Remarque : la liste retournee ne peut pas contenir plus de quatre voisins
        
        @param c : cellule a utiliser (format (x, y))0
        @return : liste des cellules accessibles de c (format [(x1, y1), (x2, y2), ...])
        """
        
        # on initialise la liste des cellules accessible pour les stocker
        cell_accessible = []
        
        # on parcourt les voisins de c
        for voisin in self.neighbors[c]:
            # si il n'y a pas de mur entre c et le voisin
            if voisin not in self.get_walls():
                # on ajoute le voisin a la liste
                cell_accessible.append(voisin)
        
        # on retourne la liste des cellules accessibles
        return cell_accessible

    
    def get_cells(self)-> list:
        """
        Methode d'instance Maze permettant de recuperer la liste de toutes les cellules de la grille du labyrinthe.
        
        @return : liste des cellules dans la grille du labyrinthe.
        """
        
        L = []
        for i in range(laby.height):
            for j in range(laby.width):
                L.append((i,j))
        return L
    
    
    def gen_btree(height,width):
        """
        Methode d'instance Maze permettant de generer un labyrinthe parfait de taille height x width en utilisant
        l'algorithmne de generation par arbre binaire. Le principe est d'initialiser un labyrinthe plein (avec tout
        les emplacements de mur occupés) et en partant de la cellule la plus en haut a gauche, supprimer aleatoirement
        le mur EST ou SUD, (s'il n'en possede qu'un, on supprime celui ci et s'il n'en possede pas, on ne supprime rien).
        
        @param height : hauteur du labyrinthe a generer
        @param width : largeur du labyrinthe a generer
        @return : le labyrinthe genere
        """
        
        # on cree un labyrinthe plein (contenant tous les murs possibles)
        laby = Maze(height, width, False)
        
        # on parcourt le labyrinthe
        for i in range(laby.height):
            for j in range(laby.width):
                # soit la cellule de coordonnees (i,j)
                c1 = (i,j)
                
                voisins_EST = None
                # si la cellule peut avoir un voisin EST
                if j<width-1:
                    # soit la cellule voisin EST de coordonnees (i,j+1)
                    voisins_EST = (i,j+1)
                voisins_SUD = None
                # si la cellule peut avoir un voisin SUD
                if i<height-1:
                    # soit la cellule voisin SUD de coordonnees (i+1,j)
                    voisins_SUD = (i+1,j)
                
                # si la cellule c1 possede un voisin SUD et EST
                if voisins_SUD != None and voisins_EST != None:
                    # si il y a un mur entre voisin SUD et c1 et un mur entre voisin EST et c1
                    if not voisins_EST in laby.get_reachable_cells(c1) and not voisins_SUD in laby.get_reachable_cells(c1):
                        # on supprime aleatoirement un mur
                        aleatoire = randint(1,2)
                        if aleatoire == 1: # supprimer le mur EST
                            laby.remove_wall(c1,voisins_EST)
                        else: # supprimer le mur SUD
                            laby.remove_wall(c1,voisins_SUD)
                
                # si le mur EST existe mais pas le mur SUD, supprimer le mur EST
                elif voisins_EST != None and not voisins_EST in laby.get_reachable_cells(c1): 
                    laby.remove_wall(c1,voisins_EST)
                
                # si le mur SUD existe mais pas le mur EST, supprimer le mur SUD
                elif voisins_SUD != None and not voisins_SUD in laby.get_reachable_cells(c1):
                    laby.remove_wall(c1,voisins_SUD)
        
        # on renvoie le labyrinthe
        return laby
    
    
    def gen_sidewinder(height,width):
        """
        Methode d'instance Maze permettant de generer un labyrinthe parfait de taille height x width en utilisant
        l'algorithmne de generation sidewinder. Le principe est d'initialiser un labyrinthe plein (avec tout
        les emplacements de mur occupés) et de raisonner ligne par ligne de cellule. Pour chaque cellule d'une ligne,
        si on peut casser le mur EST, on tire a 50% pour savoir si on casse le mur, sinon on casse le mur SUD (s'il
        existe) d'une cellule tiree au hasard sur toutes les cellules deja parcourues sur la ligne depuis le dernier
        mur SUD casse.
        
        Remarque : ce mode de generation est similaire a celui de la methode gen_btree.
        
        @param height : hauteur du labyrinthe a generer
        @param width : largeur du labyrinthe a generer
        @return : le labyrinthe genere
        """
        
        # import de randint
        from random import randint
        
        # on cree un labyrinthe plein (contenant tous les murs possibles)
        laby = Maze(height, width)
        
        # on parcourt les lignes du labyrinthe
        for i in range(height-1):
            # on initialise une sequence qui stocke les cellules d'une ligne
            sequence = []
            # on parcourt la ligne 
            for j in range(width):
                # soit la cellule cell de coordonnees (i,j)
                cell = (i,j)
                # Ajouter la cellule à la séquence
                sequence.append(cell)
                
                voisins_EST = None
                # si la cellule peut avoir un voisin EST
                if j<width-1:
                    # soit la cellule voisin EST de coordonnees (i,j+1)
                    voisins_EST = (i,j+1)
                voisins_SUD = None
                # si la cellule peut avoir un voisin SUD
                if i<height-1:
                    # soit la cellule voisin SUD de coordonnees (i+1,j)
                    voisins_SUD = (i+1,j)
                
                # tirage aleatoire
                aleatoire = randint(1,2)
                
                if aleatoire == 1 and voisins_EST != None:
                    # on casse le mur EST de la cellule 
                    laby.remove_wall(cell,voisins_EST) 
                    
                elif aleatoire == 2 and voisins_SUD != None:
                    # on tire une cellule de la sequence au hasard
                    cell_random = sequence[randint(0,len(sequence)-1)]
                    # on recupere ses coordonnees
                    x,y = cell_random
                    # on casse le mur SUD de la cellule
                    laby.remove_wall(cell_random,(x+1,y)) 
                    sequence = [] # On reinitialise la sequence
            
            
            # on ajoute la dernière cellule à la sequence
            sequence.append(cell)
            # on tire au sort une cellule de la sequence
            cell_random = sequence[randint(0,len(sequence)-1)]
            # on recupere ses coordonnees
            x,y = cell_random
            # on casse son mur SUD
            laby.remove_wall(cell_random,(x+1,y)) 
        
        # on casse tout les murs EST de la derniere ligne
        for i in range(width-1):
            laby.remove_wall((height-1,i),(height-1,i+1))
        
        # on renvoie le labyrinthe
        return laby
    
    
    def gen_fusion(height,width):
        """
        Methode d'instance Maze permettant de generer un labyrinthe parfait de taille height x width en utilisant
        l'algorithmne de generation par fusion des chemins. Le principe est d'initialiser un labyrinthe plein (avec 
        tout les emplacements de mur occupés), puis de labeliser avec un identifiant chacune des cellules du labyrinthe.
        On releve ensuite la liste des murs que l'on shuffle (librairie random). Pour chaque mur de la liste, on
        verifie le label des deux cellules, si ce dernier est different, on supprime le mur et transmet le label de
        l'une des deux cellules a l'autre.
        
        @param height : hauteur du labyrinthe a generer
        @param width : largeur du labyrinthe a generer
        @return : le labyrinthe genere
        """
        
        # import de shuffle
        from random import shuffle
        
        # on cree un labyrinthe plein (contenant tous les murs possibles)
        laby = Maze(height, width)
        
        # on cree un tableau 2d aux dimensions du labyrinthe pour y stocker les labels
        label = [[width * i + j for j in range(width)] for i in range(height)]
        
        # on extrait la liste de tous les murs et on les melange 
        murs = laby.get_walls() 
        shuffle(murs)
        
        # on parcourt la liste murs
        for mur in murs:
            # si les deux cellules separees par le mur n'ont pas le meme label
            if label[mur[0][0]][mur[0][1]] != label[mur[1][0]][mur[1][1]]:
                # on supprime le mur
                laby.remove_wall(mur[0], mur[1])
                
                # on donne le label de la premiere cellule a la seconde et a toutes les cellules avec le meme label
                label_temp = label[mur[1][0]][mur[1][1]]
                for i in range(height):
                    for j in range(width):
                        if label[i][j] == label_temp:
                            label[i][j] = label[mur[0][0]][mur[0][1]]
                label[mur[1][0]][mur[1][1]] = label[mur[0][0]][mur[0][1]]
        
        # on renvoie le labyrinthe
        return laby
    
    
    def gen_exploration(height,width):
        """
        Methode d'instance Maze permettant de generer un labyrinthe parfait de taille height x width en utilisant
        l'algorithmne de generation par exploration. Le principe est de fonctionner avec une pile, on choisis d'abord
        une cellule au hasard a marquer, on ajoute la cellule en haut de la pile. Ensuite, tant que l'on a des
        cellules dans la pile, on depile une cellule, on prends au hasard un de ses voisins non marques (s'il lui
        en reste) on casse le mur entre ce voisin et la cellule depilee, puis on empile la cellule depilee (si on 
        a casse un mur) et on marque puis empile le voisin fraichement explorer.
        
        @param height : hauteur du labyrinthe a generer
        @param width : largeur du labyrinthe a generer
        @return : le labyrinthe genere
        """
        
        # import de la methode randint
        from random import randint
        
        # on cree un labyrinthe plein (contenant tous les murs possibles)
        laby = Maze(height, width)
        
        # Choisir une cellule au hasard
        cell = (randint(0,height-1),randint(0,width-1))
        # Marquer cette cellule comme etant visitee
        cell_visite = [cell]
        # Mettre cette cellule dans sur une pile
        pile = [cell]
        
        # Tant que la pile n’est pas vide 
        while len(pile) > 0:
            # depiller une cellule
            cell = pile[len(pile)-1]
            del pile[len(pile)-1]
            
            voisins = laby.get_contiguous_cells(cell)
            voisin_pas_visite = []
            
            for i in voisins:
                if i not in cell_visite:
                    # prendre les cellules voisines qui n'ont pas ete visite 
                    voisin_pas_visite.append(i)
            
            # Si cette cellule a des voisins qui n’ont pas encore ete visites
            if len(voisin_pas_visite) > 0: 
                # La remettre sur la pile
                pile.append(cell)
                
                # Choisir au hasard l’une de ses cellules contigues qui n’a pas ete visitee
                cell_voisine = voisin_pas_visite[randint(0,len(voisin_pas_visite)-1)]
                # Casser le mur entre la cellule (celle qui a ete dépilee) et celle qui vient d’etre choisie
                laby.remove_wall(cell,cell_voisine)
                # Marquer la cellule qui vient d’etre choisie comme visitee
                cell_visite.append(cell_voisine)
                # Mettre la cellule voisines sur la pile
                pile.append(cell_voisine)
                
        return laby
    
    
    def gen_wilson(height, width):
        import random
        laby = Maze(height, width) # un labyrinthe plein (contenant tous les murs possibles)
        cells = laby.get_cells() # récupère la liste de toutes les cellules

        # on choisit une cellule au hasard pour commencer
        start_cell = random.choice(cells)
        cells.remove(start_cell)

        # on ajoute la cellule choisie à la liste des cellules visitées
        visited = [start_cell]

        # tant qu'il reste des cellules non visitées
        while cells:
            # on choisit une cellule non visitée au hasard
            c = random.choice(cells)

            # on crée une liste de cellules visitées vides pour stocker le chemin qui va être parcouru
            path = [c]

            # on parcourt des cellules au hasard jusqu'à atteindre une cellule visitée
            while c in cells:
                # on choisit un voisin de c au hasard
                next_c = random.choice(laby.get_contiguous_cells(c))

                # si le voisin choisi est dans le chemin parcouru, on élimine toutes les cellules du chemin qui viennent après le voisin
                if next_c in path:
                    idx = path.index(next_c)
                    path = path[:idx+1]

                # sinon on ajoute la cellule au chemin parcouru et on la marque comme visitée
                else:
                    path.append(next_c)
                    visited.append(next_c)

                # la cellule suivante devient la nouvelle cellule courante
                c = next_c

            # on supprime toutes les cellules visitées du chemin
            for cell in path:
                if cell in cells:
                    cells.remove(cell)

            # on ouvre les murs entre les cellules du chemin
            for i in range(len(path)-1):
                laby.remove_wall(path[i], path[i+1])

        return laby
    
    """
    
    def overlay(self, content=None):
        
        Rendu en mode texte, sur la sortie standard, \
        d'un labyrinthe avec du contenu dans les cellules
        Argument:
            content (dict) : dictionnaire tq content[cell] contient le caractère à afficher au milieu de la cellule
        Retour:
            string
        
        if content is None:
            content = {(i,j):' ' for i in range(self.height) for j in range(self.width)}
        else:
            # Python >=3.9
            #content = content | {(i, j): ' ' for i in range(
            #    self.height) for j in range(self.width) if (i,j) not in content}
            # Python <3.9
            new_content = {(i, j): ' ' for i in range(self.height) for j in range(self.width) if (i,j) not in content}
            content = {**content, **new_content}
        txt = r""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += " "+content[(0,j)]+" ┃" if (0,j+1) not in self.neighbors[(0,j)] else " "+content[(0,j)]+"  "
        txt += " "+content[(0,self.width-1)]+" ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += " "+content[(i+1,j)]+" ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else " "+content[(i+1,j)]+"  "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"
        return txt
  
    """

In [25]:
laby = Maze.gen_wilson(15,15)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃           ┃       ┃                   ┃       ┃           ┃
┣   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ┫
┃                   ┃       ┃           ┃           ┃       ┃
┣   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ┫
┃   ┃       ┃       ┃       ┃               ┃   ┃   ┃       ┃
┣━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋   ┫
┃       ┃   ┃       ┃           ┃   ┃       ┃           ┃   ┃
┣   ╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ┫
┃   ┃   ┃   ┃           ┃       ┃       ┃   ┃   ┃       ┃   ┃
┣━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ┫
┃               ┃   ┃   ┃           ┃           ┃           ┃
┣   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━┫
┃       ┃       ┃           ┃   ┃   ┃       ┃       ┃       ┃
┣━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ┫
┃           ┃   ┃       ┃               ┃       ┃       ┃   ┃
┣   ╋   